<a href="https://colab.research.google.com/github/bencode/tfadv/blob/main/C1W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        for i in range(repetitions):
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', padding='same')
        
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides)
  
    def call(self, inputs):
        conv2D_0 = vars(self)['conv2D_0']
        
        x = conv2D_0(inputs)

        for i in range(1, self.repetitions):
            conv2D_i = vars(self)[f'conv2D_{i}']
            x = conv2D_i(x)

        max_pool = self.max_pool(x)
        
        return max_pool

In [4]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        self.block_a = Block(64, 3, 2)
        self.block_b = Block(128, 3, 2)
        self.block_c = Block(256, 3, 3)
        self.block_d = Block(512, 3, 3)
        self.block_e = Block(512, 3, 3)

        self.flatten = tf.keras.layers.Flatten()

        self.fc = tf.keras.layers.Dense(256, activation='relu')

        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [5]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 
vgg = MyVGG(num_classes=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=10)

Shuffling and writing examples to data/cats_vs_dogs/4.0.0.incompleteHEO3NT/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to data/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Epoch 1/10
727/727 [==============================] - 71s 72ms/step - loss: 0.6825 - accuracy: 0.5692
Epoch 2/10
727/727 [==============================] - 50s 68ms/step - loss: 0.6455 - accuracy: 0.6374
Epoch 3/10
727/727 [==============================] - 50s 68ms/step - loss: 0.6169 - accuracy: 0.6672
Epoch 4/10
727/727 [==============================] - 50s 68ms/step - loss: 0.6000 - accuracy: 0.6802
Epoch 5/10
727/727 [==============================] - 50s 69ms/step - loss: 0.5888 - accuracy: 0.6889
Epoch 6/10
727/727 [==============================] - 50s 68ms/step - loss: 0.5801 - accuracy: 0.6947
Epoch 7/10
727/727 [==============================] - 50s 69ms/step - loss: 0.5727 - accuracy: 0.7016
Epoch 8/10
727/727 [==============================] - 50s 68ms/step - loss: 0.5660 - accuracy: 0.7077
Epoch 9/10
727/727 [==============================] - 49s 68ms/step - lo